## 测试代理是否正常

In [1]:
# 测试代理
import os
import requests

# 设置代理环境变量
#os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
#os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
#os.environ['ALL_PROXY'] = 'socks5://127.0.0.1:7891'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 测试代理连接
try:
    response = requests.get('https://huggingface.co', timeout=10)
    print("✅ HuggingFace 连接成功，状态码:", response.status_code)
    print("Test server is ubuntu22.04 GPU 2080Ti 22G")
except Exception as e:
    print("连接失败:", e)

# 设置 HuggingFace 缓存路径
os.environ['HF_HOME'] = '/home/KevinLiangX/Codes/LLM-quickstart-main/hf'
os.environ['HF_HUB_CACHE'] = '/home/KevinLiangX/Codes/LLM-quickstart-main/hf_hu'


# 服务器环境 ubuntu22.04 GPU 2080Ti 22G

连接失败: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x76fc9caeb2b0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


In [2]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO 

In [3]:
# 定义全局变量和参数
model_name_or_path = '/home/KevinLiangX/Codes/LLM-quickstart-main/quantization/ChatGLM3-6b'  # 模型ID或本地路径
train_data_path = 'data/zhouyi_dataset_20240118_152413.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,周易的坤卦讲述了什么？,"坤卦是周易中的一卦，代表大地的形势，象征顺顺利利，顺应天道。它由两个坤卦叠加而成，全为阴卦，具有纯阴性质。在卜问中，坤卦预示着利于雌马的贞正之象，吉利的事情会发生。同时，出行需谨慎，起初可能会迷失方向，但最终会找到主人，带来吉利。前往西南会得到朋友，前往东北会失去朋友。在安定的状态下，能够得到吉祥。\n\n坤卦的形势平铺舒展，代表地道生育抚养万物，依循天时地利，象征以厚德载物。在决策上，应该顺从运势，守正静待，不宜急进，须以静制动为宜。事业发展上，要注重内心修养，广纳众意，以柔克刚为原则，力求和谐共生。经商求名上，同样要稳健行事，不宜冒险急进，在合作中共同完成事业。\n\n综上所述，坤卦意味着安稳、柔顺、温和、顺应天道，需要以厚德载物，以守正静待为策略，以诚信待人，收敛于己，力求温和和谐。""\n\nformat:""卦名:坤卦\n解释:坤卦是周易中的一卦，代表大地的形势，象征顺顺利利，顺应天道。它由两个坤卦叠加而成，全为阴卦，具有纯阴性质。在卜问中，坤卦预示着利于雌马的贞正之象，吉利的事情会发生。同时，出行需谨慎，起初可能会迷失方向，但最终会找到主人，带来吉利。前往西南会得到朋友，前往东北会失去朋友。在安定的状态下，能够得到吉祥。\n\n坤卦的形势平铺舒展，代表地道生育抚养万物，依循天时地利，象征以厚德载物。在决策上，应该顺从运势，守正静待，不宜急进，须以静制动为宜。事业发展上，要注重内心修养，广纳众意，以柔克刚为原则，力求和谐共生。经商求名上，同样要稳健行事，不宜冒险急进，在合作中共同完成事业。\n\n综上所述，坤卦意味着安稳、柔顺、温和、顺应天道，需要以厚德载物，以守正静待为策略，以诚信待人，收敛于己，力求温和和谐。"
1,周易中的蒙卦含义是什么？,在周易中，蒙卦代表着通泰的格局。卦象为艮上坎下，意味着山下有泉，波涌的情形。《象辞》中解释说，君子观此卦象，取法于一往无前的山泉，从而以果敢坚毅的行动来培养自身的品德。\n\n蒙卦的核心哲学是：这是一个启蒙的过程，开始时可能面临迷惑和混乱，需要忍耐等待时机，倾听他人意见，通过果敢的行动来扭转局面。\n\n蒙卦的传统解卦中指出，初时迷惑不知方向，需要须忍耐待机而动，凡事多听取别人意见，则运可通。因此，在事业、经商、求名、婚恋等方面应当注重实际，接受严格教育和考验，培养果敢坚毅的品质，始能取得成功。
2,"""讼卦和教育启蒙有什么联系？",在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。\n\n讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。\n\n讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。\n\n对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。\n\n讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。
3,"{\n ""content"": ""屯卦"",\n ""summary"": ""在周易中，屯卦代表着大吉大利、吉利的占卜，但不利于出门，有利于建国封侯。其卦象为坎上震下，代表着困难和动而逢险的象征，需要刚毅果敢的态度才能取得吉利。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。"",\n ""interpretation"": ""屯卦的核心哲学是：万物始生，开始困难；先劳后逸，苦尽甘来。得此卦者，身处困境，宜守不宜进，须多加辛苦努力，排除困难，方可通达，有初难后解之象。在事业上，起初多有不利，要知难而进，小心翼翼，勇往直前，灵活机动，可望获得大的成功，时机到来时一定要抓住，却也不得操之太急，且仍有困难，务必有他人相助，故平时应多施恩惠。在经商方面，创业初期步履艰难，多有挫折。坚定信念最重要，不要为表面现象所迷惑，应积极进取，行动果断，打开出路。若仍无法摆脱困境，则应退守保全，等待机会，再展宏图。而在婚恋中，好事多磨，忠贞纯洁，大胆追求，能够成功，婚姻美满。""\n}\n``是怎样的一个卦象？","{\n ""content"": ""屯卦"",\n ""summary"": ""在周易中，屯卦代表着大吉大利、吉利的占卜，但不利于出门，有利于建国封侯。其卦象为坎上震下，代表着困难和动而逢险的象征，需要刚毅果敢的态度才能取得吉利。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。"",\n ""interpretation"": ""屯卦的核心哲学是：万物始生，开始困难；先劳后逸，苦尽甘来。得此卦者，身处困境，宜守不宜进，须多加辛苦努力，排除困难，方可通达，有初难后解之象。在事业上，起初多有不利，要知难而进，小心翼翼，勇往直前，灵活机动，可望获得大的成功，时机到来时一定要抓住，却也不得操之太急，且仍有困难，务必有他人相助，故平时应多施恩惠。在经商方面，创业初期步履艰难，多有挫折。坚定信念最重要，不要为表面现象所迷惑，应积极进取，行动果断，打开出路。若仍无法摆脱困境，则应退守保全，等待机会，再展宏图。而在婚恋中，好事多磨，忠贞纯洁，大胆追求，能够成功，婚姻美满。"
4,"""讼卦涉及哪些哲学思想？",在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。\n\n讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。\n\n讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。\n\n对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。\n\n讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

In [8]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/160 [00:00<?, ? examples/s]

In [9]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [10]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

In [11]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [12]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [13]:
target_modules

['query_key_value']

In [14]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [15]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


In [16]:
train_epochs = 3
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}"

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [18]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [19]:
trainer.train()

/home/KevinLiangX/Codes/LLM-quickstart-main/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.594100
2,4.048900
3,3.090000
4,3.376300
5,3.544800
6,2.610500
7,2.660200
8,3.165400
9,2.275000
10,2.292100


/home/KevinLiangX/Codes/LLM-quickstart-main/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/KevinLiangX/Codes/LLM-quickstart-main/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/hom

TrainOutput(global_step=60, training_loss=0.6936044594350581, metrics={'train_runtime': 358.8741, 'train_samples_per_second': 1.338, 'train_steps_per_second': 0.167, 'total_flos': 9433079239507968.0, 'train_loss': 0.6936044594350581, 'epoch': 3.0})

In [20]:
trainer.model.save_pretrained(output_dir)

In [21]:
# 使用qlora后的模型
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

In [24]:
# 使用原始模型
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)

In [25]:
print(response)

乾卦是《易经》中的一个卦象，它是由六个阳爻（表示阳、积极、明、刚等特性）和一条阴爻（表示阴、消极、暗、柔等特性）组成的。乾卦象征着天、干燥、强、积极等特性。在八宫图（后天八卦）中，乾卦位于北方，与事业、努力、坚定、果敢等特性相关。

乾卦的卦辞是：“元、亨、利、永。”意味着这个卦象具有绝对的、顺利的、有益的、永恒的特性。在《易经》中，乾卦主要阐述阳刚之气、阳性的特性和功能。乾卦鼓励人们要积极向上、坚定信念、勇往直前、追求成功。

乾卦的变化过程是：初爻为阳爻，二爻为阳爻，三爻为阳爻，四爻为阳爻，五爻为阳爻，六爻为阳爻，由初爻变至六爻均为阳爻，形成一个完整的阳刚之卦。这个变化过程代表了一种不断追求进步、发展、壮大的精神。


In [26]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的一个卦象，它是由三个阳爻和三个阴爻组成的。这个卦象象征着诉讼、争端、诉讼程序、法律、公正等。在八宫图（后天八卦）中，讼卦位于西方，与法律、制度、规则、竞争等特性相关。

讼卦的卦辞是：“天罚、好古、古之道、利见大人。”意味着这个卦象具有天空惩罚、喜好古代制度、古代智慧、有利于出现伟大人物等特性。

从哲学角度来看，讼卦反映了人们在处理争端和诉讼时的态度和行为。它告诉我们在面对诉讼和争端时，要遵循古代的制度和智慧，保持公正和公平，有利于出现伟大的人物来解决争端。

需要注意的是，虽然讼卦具有这些特性，但它并不是一个积极的、乐观的卦象。相反，它提醒我们要谨慎处理争端和诉讼，遵循法律和制度，避免过度的争斗和冲突。


In [27]:
# 使用微调后的模型
from peft import PeftModel, PeftConfig

#epochs = 3
# timestamp = "20240118_164514"
#timestamp = "20240225_222843"

peft_model_path = "/home/KevinLiangX/Codes/LLM-quickstart-main/chatglm/models/home/KevinLiangX/Codes/LLM-quickstart-main/quantization/ChatGLM3-6b-epoch3"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag="ChatGLM3-6b-epoch"

In [28]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [29]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，它由六个阳爻组成，象征着刚健强劲的特性。在卜筮时，乾卦预示着顺利，强大的力量和吉祥如意。同时，SER.bs的AI解释，在运势上，选择对了事业和合作伙伴，可以带来积极的影响。在工作和 business 上，刚直的个性可能会让情况变得困难，因此需要更多的耐心和合作。在决策上，需要坚持正道，积极向上，不可冒进。在人际关系上，需要强调互相信任和尊重，更多的沟通和交流。


微调后（ChatGLM3-6b-epoch）：
[gMASK]sop 解释下乾卦是什么？ 

 乾卦是周易中的一卦，代表天，它由六个阳爻组成，象征着刚健强劲的特性。在周易中，阳卦代表积极、主动、刚强、强劲的特性，而阴卦则代表消极、被动、柔弱、柔和的特性。

乾卦的卦辞描述了阳刚之性的特点，如“元、亨、利、贞”等，表示万事以元为始，亨通无阻，以利待人，刚强不屈。在周易中，阳卦代表积极、主动、刚强、强劲的特性，而阴卦则代表消极、被动、柔弱、柔和的特性。

乾卦的阳刚特性反映了阳爻的强大力量，同时也体现了刚强不屈的性格。在决策中，应该充分利用阳爻的力量，刚强不屈，坚持到底。在面临困难时，应该保持冷静，以柔克刚，以柔顺求得解决办法。

在解卦时，起初没有遇到困难，但后来会面临困难，需要坚贞刚强，刚强不屈，以柔克刚，最终能够解决问题。在决策中，应该充分利用阳爻的力量，刚强不屈，坚持到底。在面临困难时，应该保持冷静，以柔克刚，以柔顺求得解决办法。


In [30]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，由上卦坎（水）和下卦乾（天）组成，上卦为天，下卦为水，象征着天和水的相激相磨，象征着矛盾和天理之间的冲突。讼卦意味着天理和人道之间存在冲突，提醒人们要谨慎行事，遵循天理，克服困难。

讼卦的取义在于：天理要求人们遵守一定的规则，和人相互尊重。然而，人道有时与天理相悖，人们可能会为了自己的利益而违反天理。因此，讼卦提醒人们要警惕矛盾和冲突，要公正、刚正，不能让邪恶势力得逞。

讼卦的战争之象为天上有风，有雷，有雨，有云，寓意着矛盾和冲突的激烈和复杂。然而，最终天象会平息， Rain止于乎，象征着矛盾和天理之间的冲突最终会得到解决。

讼卦的时运为：卦中动卦为乾，固定卦为坎，时运为正，可以得到众人的帮助。然而，由于天理和人道之间的矛盾，最终可能无法得到众人的帮助。

讼卦的解析需要综合考虑天时、地利、人和等因素，强调人在面对矛盾和冲突时，要刚正不阿，坚持原则，不能让邪恶势力得逞。在处理纠纷和冲突时，需要谨慎、冷静，尽量化解矛盾，寻求共同利益。


微调后（ChatGLM3-6b-epoch）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个由上卦坎水和下卦乾运动的卦象，上卦为坎卦，下卦为乾卦。在卜筮中，讼卦预示着双方争斗，互不相让，互为敌手。

讼卦的核心哲学是：

* 双方为争夺利益和地位而争执
* 双方都欲争执之极，不欲争执之终
* 双方都未能认识到双方争斗的后果
* 双方都未能找到合适的解决方案

讼卦的启示是：

* 双方应该保持冷静，以和平的方式解决争端
* 双方应该寻求中立的裁判者，以公正的态度判断
* 双方应该认识到争斗的后果，避免发生冲突
* 双方应该以和平、合作、让步为原则，以达成共识

讼卦的运势：

* 当前运势为争执和冲突
* 双方未能相互理解，容易发生争执
* 需要保持冷静，避免发生冲突
* 适合寻求第三方的帮助，以解决争端
* 适合寻求宗教、哲学、哲学等领域的支持

讼卦的运势影响：

* 争执和冲突可能会对事业和学业产生负面影响
* 情绪波动较大，容易影响人际关系
* 需要特别注意身体和心理的健康
* 适合寻求专业的心理辅导和治疗

讼卦的应对策略：

* 保持冷静，以和平的方式解决争端
* 寻求中立的裁判者，以公正的态度判断
* 保持谦虚，尊重对方观点
* 避免过激行为

In [31]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是周易中的一卦，由坎卦（水）上承坤卦（地）组成，代表军队和指挥军情的卦象。在卜问中，师卦预示着占卜者将得到众人的支持，并可以统帅众人。

师卦的核心哲学是：

师者，众也。

这个卦象表明，师卦象征着众人的力量和集体智慧。在卜问中，需要顺应众人的意愿，领导者需要广泛听取他人的意见和建议，以便做出正确的决策。

在师卦中，领导者需要保持警惕，避免盲目自信，不要被情绪左右，保持冷静和理智。同时，需要choose合理的态度和行为，以达到最好的结果。

在人际关系方面，需要广泛结交朋友，特别是那些能够给予支持的人。在处理人际关系时，需要以诚相待，保持友好和互助的态度。

在事业和 business 上，师卦预示着成功和繁荣。在事业和 business 中，需要注重团结和合作，广泛听取他人的意见和建议，以便做出正确的决策。

在生活方面，需要注重健康和养生，保持良好的生活习惯，注重休息和调整。

总结起来，师卦预示着，在 various aspects of life, that is, in career, friendship, and personal life, one should be open-minded, listen to others' opinions, and be aware of the power of collective wisdom, and make rational choices to achieve success.


微调后（ChatGLM3-6b-epoch）：
[gMASK]sop 师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人